In [2]:
import os 
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
from progressbar import ProgressBar
import math
import numpy as np
import pickle 

In [2]:
folders = []

for i in os.listdir('/content/drive/MyDrive/Diplomacy/game-states-batch1/'):
  folders.append(['/content/drive/MyDrive/Diplomacy/game-states-batch1/', i])

In [7]:
files = []
pbar = ProgressBar()

for folder in pbar(folders):
  for file in os.listdir(folder[0] + folder[1]):
    if 'payoff' in file:
      files.append([folder[0], folder[1], file])

100% (3829 of 3829) |####################| Elapsed Time: 0:03:49 Time:  0:03:49


In [10]:
file_df = pd.DataFrame(files, columns = ['parent_dir', 'game_id', 'file'])

In [11]:
file_df

,parent_dir,game_id,file
0,/content/drive/MyDrive/Diplomacy/game-states-b...,111483,20 - S1906M - alliance_attacker_payoffs.csv
1,/content/drive/MyDrive/Diplomacy/game-states-b...,111483,21 - F1906M - alliance_attacker_payoffs.csv
2,/content/drive/MyDrive/Diplomacy/game-states-b...,111483,1 - S1901M - alliance_attacker_payoffs.csv
3,/content/drive/MyDrive/Diplomacy/game-states-b...,111483,2 - F1901M - alliance_attacker_payoffs.csv
4,/content/drive/MyDrive/Diplomacy/game-states-b...,111483,4 - S1902M - alliance_attacker_payoffs.csv
...,...,...,...
65329,/content/drive/MyDrive/Diplomacy/game-states-b...,112901,52 - S1914M - alliance_attacker_payoffs.csv
65330,/content/drive/MyDrive/Diplomacy/game-states-b...,112901,53 - F1914M - alliance_attacker_payoffs.csv
65331,/content/drive/MyDrive/Diplomacy/game-states-b...,112901,54 - S1915M - alliance_attacker_payoffs.csv
65332,/content/drive/MyDrive/Diplomacy/game-states-b...,112901,55 - F1915M - alliance_attacker_payoffs.csv


In [12]:
game_id = file_df['game_id'].iloc[0]

In [13]:
game_id

'111483'

In [17]:
game_files = file_df[file_df['game_id'] == game_id].reset_index(drop=True)

In [110]:
payoff_dict = {}
pbar = ProgressBar()

for game_id in pbar(file_df['game_id'].unique()):
  
  game_files = file_df[file_df['game_id'] == game_id].reset_index(drop=True)

  for i in game_files.index:
      row = game_files.iloc[i]
      phase = row['file'].split('-')[1].strip()
      phase_payoffs = pd.read_csv(row['parent_dir'] + game_id + '/' + row['file'])

      phase_payoffs = phase_payoffs[phase_payoffs['power'].notnull()].reset_index(drop=True)

      for power in phase_payoffs['power'].to_list():
        
        if power not in payoff_dict.keys():
          payoff_dict[power] = {}

        if phase not in payoff_dict[power].keys():
          payoff_dict[power][phase] = {}

        cooperates = phase_payoffs[phase_payoffs['power'] == power]['cooperates'].iloc[0]
        if pd.isnull(cooperates):
          cooperates = frozenset()
        if len(cooperates) > 0:
          cooperates = frozenset(set(eval(cooperates)))

        attacks = phase_payoffs[phase_payoffs['power'] == power]['attacks'].iloc[0]
        if pd.isnull(attacks):
          attacks = frozenset()
        if len(attacks) > 0:
          attacks = frozenset(set(eval(attacks)))

        if (cooperates, attacks) not in payoff_dict[power][phase].keys():
          payoff_dict[power][phase][(cooperates, attacks)] = []

        payoff = phase_payoffs[phase_payoffs['power'] == power]['payoff'].iloc[0]
        payoff_dict[power][phase][(cooperates, attacks)].append(payoff)

100% (3810 of 3810) |####################| Elapsed Time: 3:13:36 Time:  3:13:36


In [111]:

with open('/content/drive/MyDrive/Diplomacy/game-states-batch1-payoff_dictionary.pkl', 'wb') as f:
    pickle.dump(payoff_dict, f)

In [5]:
with open('/content/drive/MyDrive/Diplomacy/game-states-batch1-payoff_dictionary.pkl', 'rb') as f:
    payoff_dict = pickle.load(f)

In [ ]:
# Get full distribution of payoffs

pbar = ProgressBar()
all_payoffs = []

for power in pbar(payoff_dict.keys()):
  for phase in payoff_dict[power].keys():
    for coop, attack in payoff_dict[power][phase].keys():
      payoff_df = pd.DataFrame(payoff_dict[power][phase][(coop, attack)], columns = ['payoff'])
      payoff_df['power'] = power
      payoff_df['phase'] = phase
      payoff_df['coop'] = str(coop)
      payoff_df['attack'] = str(attack)
      all_payoffs.append(payoff_df)

all_payoff_df = pd.concat(all_payoffs, ignore_index=True)

all_payoff_df['coop'] = all_payoff_df['coop'].apply(lambda x: set(eval(x)))
all_payoff_df['attack'] = all_payoff_df['attack'].apply(lambda x: set(eval(x)))

all_payoff_df['phase-month'] = all_payoff_df['phase'].apply(lambda x: 3 if x[0] == 'S' else 9)
all_payoff_df['year'] = all_payoff_df['phase'].apply(lambda x: x[1:-1])
all_payoff_df = all_payoff_df.sort_values(by=['year', 'phase-month', 'power'])
all_payoff_df.to_csv('/content/drive/MyDrive/Diplomacy/all_game_payoffs.csv', index=False)

In [20]:
# Take mean of payoff for aggregated version


payoffs = []

for power in payoff_dict.keys():
  for phase in payoff_dict[power].keys():
    for coop, attack in payoff_dict[power][phase].keys():
      payoffs.append([power, phase, coop, attack, len(payoff_dict[power][phase][(coop, attack)]), np.mean(payoff_dict[power][phase][(coop, attack)])])

In [21]:
df = pd.DataFrame(payoffs, columns = ['power', 'phase', 'cooperate', 'attack', 'n_obs', 'mean_payoff'])

In [35]:
df['phase-month'] = df['phase'].apply(lambda x: 3 if x[0] == 'S' else 9)
df['year'] = df['phase'].apply(lambda x: x[1:-1])

In [37]:
df = df.sort_values(by=['year', 'phase-month', 'power'])

In [38]:
df.to_csv('/content/drive/MyDrive/Diplomacy/payoffs.csv', index=False)